In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint

# llm = HuggingFaceEndpoint(
#     repo_id="artificialguybr/llama3-8b-sql-create-context",
#     task="text-generation",
#     max_new_tokens=512,
#     do_sample=False,
#     repetition_penalty=1.03,
# )

In [2]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
import os
import torch

# max_split_size_mb 설정
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# GPU 캐시 비우기
torch.cuda.empty_cache()

# 모델 로드
llm = HuggingFacePipeline.from_model_id(
    model_id="artificialguybr/llama3-8b-sql-create-context",
    task="text-generation",
    device=0,
    pipeline_kwargs={
        "max_new_tokens": 512,  # 토큰 수 줄이기
        "top_k": 5,
        "temperature": 0.1
    },
    model_kwargs={
        "torch_dtype": torch.float16  # float16으로 메모리 사용량 줄이기
    }
)

# llm_engine_hf = ChatHuggingFace(llm=llm)
# response = llm_engine_hf.invoke("Hugging Face is")
# print(response)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 

In [ ]:
llm.invoke('hello, tell me a joke.')

In [ ]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

llm = HuggingFacePipeline.from_model_id(
    model_id="artificialguybr/llama3-8b-sql-create-context",
    task="text-generation",
    device=0,
    pipeline_kwargs={
        "max_new_tokens": 4000,
        "top_k": 5,
        "temperature": 0.1
    },
)

llm_engine_hf = ChatHuggingFace(llm=llm)
response = llm_engine_hf.invoke("Hugging Face is")
print(response)

In [ ]:
chat_model.invoke('hello')